In [1]:
df = spark.read.csv('/FileStore/tables/hour.csv',header = 'True')
df.show()

+-------+----------+------+---+----+---+-------+-------+----------+----------+----+------+----+---------+------+----------+---+
instant| dteday|season| yr|mnth| hr|holiday|weekday|workingday|weathersit|temp| atemp| hum|windspeed|casual|registered|cnt|
+-------+----------+------+---+----+---+-------+-------+----------+----------+----+------+----+---------+------+----------+---+
 1|2011-01-01| 1| 0| 1| 0| 0| 6| 0| 1|0.24|0.2879|0.81| 0| 3| 13| 16|
 2|2011-01-01| 1| 0| 1| 1| 0| 6| 0| 1|0.22|0.2727| 0.8| 0| 8| 32| 40|
 3|2011-01-01| 1| 0| 1| 2| 0| 6| 0| 1|0.22|0.2727| 0.8| 0| 5| 27| 32|
 4|2011-01-01| 1| 0| 1| 3| 0| 6| 0| 1|0.24|0.2879|0.75| 0| 3| 10| 13|
 5|2011-01-01| 1| 0| 1| 4| 0| 6| 0| 1|0.24|0.2879|0.75| 0| 0| 1| 1|
 6|2011-01-01| 1| 0| 1| 5| 0| 6| 0| 2|0.24|0.2576|0.75| 0.0896| 0| 1| 1|
 7|2011-01-01| 1| 0| 1| 6| 0| 6| 0| 1|0.22|0.2727| 0.8| 0| 2| 0| 2|
 8|2011-01-01| 1| 0| 1| 7| 0| 6| 0| 1| 0.2|0.2576|0.86| 0| 1| 2| 3|
 9|2011-01-01| 1| 0| 1| 8| 0| 6| 0| 1|0.24|0.2879|0.75| 0| 1| 7| 8|
 10|2011-01-01| 1| 0| 1| 9| 0| 6| 0| 1|0.32|0.3485|0.76| 0| 8| 6| 14|
 11|2011-01-01| 1| 0| 1| 10| 0| 6| 0| 1|0.38|0.3939|0.76| 0.2537| 12| 24| 36|
 12|2011-01-01| 1| 0| 1| 11| 0| 6| 0| 1|0.36|0.3333|0.81| 0.2836| 26| 30| 56|
 13|2011-01-01| 1| 0| 1| 12| 0| 6| 0| 1|0.42|0.4242|0.77| 0.2836| 29| 55| 84|
 14|2011-01-01| 1| 0| 1| 13| 0| 6| 0| 2|0.46|0.4545|0.72| 0.2985| 47| 47| 94|
 15|2011-01-01| 1| 0| 1| 14| 0| 6| 0| 2|0.46|0.4545|0.72| 0.2836| 35| 71|106|
 16|2011-01-01| 1| 0| 1| 15| 0| 6| 0| 2|0.44|0.4394|0.77| 0.2985| 40| 70|110|
 17|2011-01-01| 1| 0| 1| 16| 0| 6| 0| 2|0.42|0.4242|0.82| 0.2985| 41| 52| 93|
 18|2011-01-01| 1| 0| 1| 17| 0| 6| 0| 2|0.44|0.4394|0.82| 0.2836| 15| 52| 67|
 19|2011-01-01| 1| 0| 1| 18| 0| 6| 0| 3|0.42|0.4242|0.88| 0.2537| 9| 26| 35|
 20|2011-01-01| 1| 0| 1| 19| 0| 6| 0| 3|0.42|0.4242|0.88| 0.2537| 6| 31| 37|
+-------+----------+------+---+----+---+-------+-------+----------+----------+----+------+----+---------+------+----------+---+
only showing top 20 rows

In [2]:
print "Our dataset has %d rows." % df.count()


Our dataset has 17379 rows.

In [3]:
df = df.drop("instant").drop("dteday").drop("casual").drop("registered")
df.printSchema()


root
-- season: string (nullable = true)
-- yr: string (nullable = true)
-- mnth: string (nullable = true)
-- hr: string (nullable = true)
-- holiday: string (nullable = true)
-- weekday: string (nullable = true)
-- workingday: string (nullable = true)
-- weathersit: string (nullable = true)
-- temp: string (nullable = true)
-- atemp: string (nullable = true)
-- hum: string (nullable = true)
-- windspeed: string (nullable = true)
-- cnt: string (nullable = true)

In [4]:
df = df.select([col(c).cast("double").alias(c) for c in df.columns])

In [5]:
df.printSchema()

root
-- season: double (nullable = true)
-- yr: double (nullable = true)
-- mnth: double (nullable = true)
-- hr: double (nullable = true)
-- holiday: double (nullable = true)
-- weekday: double (nullable = true)
-- workingday: double (nullable = true)
-- weathersit: double (nullable = true)
-- temp: double (nullable = true)
-- atemp: double (nullable = true)
-- hum: double (nullable = true)
-- windspeed: double (nullable = true)
-- cnt: double (nullable = true)

In [6]:
train, test = df.randomSplit([0.7, 0.3])

In [7]:
from pyspark.ml.feature import VectorAssembler, VectorIndexer
featuresCols = df.columns
featuresCols.remove('cnt')


In [8]:
vectorAssembler = VectorAssembler(inputCols=featuresCols, outputCol="rawFeatures")
# This identifies categorical features and indexes them.
vectorIndexer = VectorIndexer(inputCol="rawFeatures", outputCol="features", maxCategories=4)
from pyspark.ml.regression import GBTRegressor
# Takes the "features" column and learns to predict "cnt"
gbt = GBTRegressor(labelCol="cnt")

from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator
paramGrid = ParamGridBuilder()\
  .addGrid(gbt.maxDepth, [2, 5])\
  .addGrid(gbt.maxIter, [10, 100])\
  .build()

In [9]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol=gbt.getLabelCol(), predictionCol=gbt.getPredictionCol())
cv = CrossValidator(estimator=gbt, evaluator=evaluator, estimatorParamMaps=paramGrid)
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[vectorAssembler, vectorIndexer, cv])
pipelineModel = pipeline.fit(train)
predictions = pipelineModel.transform(test)
predictions.select("cnt", "prediction", *featuresCols)
rmse = evaluator.evaluate(predictions)
print "RMSE on our test set: %g" % rmse

RMSE on our test set: 46.4057